In [1]:
# -*- coding: utf-8 -*-
"""main_v3_dual_evaluator.py

LangGraph Joke Generation Pipeline with Dual Evaluation (GPT-OSS + Mistral)
"""

# !pip uninstall -y transformers accelerate peft bitsandbytes trl
# !pip install -q -U bitsandbytes accelerate peft transformers trl datasets
# !pip install faiss-cpu

import os
import json
import torch
import numpy as np
from typing import List, Dict, Tuple
from dataclasses import dataclass
import pickle
from tqdm import tqdm

from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
from sentence_transformers import SentenceTransformer
import faiss

import wandb
from huggingface_hub import notebook_login

In [2]:
"""
Setup environment and imports
"""

print("🔧 Setting up environment...")

# Check for GPU
if torch.cuda.is_available():
    print(f"✓ GPU detected: {torch.cuda.get_device_name(0)}")
    print(f"  Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")
else:
    print("⚠️ No GPU detected. Will use CPU (slower)")

print(" Environment setup complete\n")

"""
LangGraph Joke Generation Pipeline - Orchestrator with Dual Evaluation
Imports and orchestrates existing implementations
"""

"""
LangGraph Joke Generation Pipeline - Orchestrator
Imports and orchestrates existing implementations
"""
from typing import TypedDict, Literal, List, Dict
from langgraph.graph import StateGraph, END
from pathlib import Path

import sys

#  set project root = folder c:\Users\Anwender\humor-project
PROJECT_ROOT = Path(__file__).resolve().parent if "__file__" in globals() else Path.cwd()

# if you run from a notebook inside some subfolder, walk up until we find "src"
while not (PROJECT_ROOT / "src").exists() and PROJECT_ROOT.parent != PROJECT_ROOT:
    PROJECT_ROOT = PROJECT_ROOT.parent

sys.path.insert(0, str(PROJECT_ROOT))         # so "import src...." works
sys.path.insert(0, str(PROJECT_ROOT / "src")) # so "import evaluation...." works if you want
print("Using PROJECT_ROOT:", PROJECT_ROOT)
# Import your existing implementations
from using_llama_with_rag import RAGHumorGenerator, RAGConfig
from gpt_joke_generation import GPT4JokeGenerator
from src.evaluation.llm_evaluator import LLMHumorScorer
from src.evaluation.mistral_evaluator import MistralHumorScorer

print(" All modules imported successfully\n")


🔧 Setting up environment...
✓ GPU detected: NVIDIA GeForce RTX 5090 Laptop GPU
  Memory: 25.65 GB
 Environment setup complete

Using PROJECT_ROOT: c:\Users\Anwender\humor-project
 All modules imported successfully



In [ ]:

# ============================================================================
# LANGGRAPH STATE DEFINITION
# ============================================================================

class JokeGenerationState(TypedDict):
    """State for the joke generation pipeline"""
    # Input
    word1: str
    word2: str

    # Generation tracking
    llama_jokes: List[str]
    gpt_jokes: List[str]
    candidate_jokes: List[str]
    joke_sources: List[str]      # "Llama" or "GPT" per joke

    # Scoring tracking
    scores: List[float]
    gpt_scores: List[float]      # Track GPT scores separately
    mistral_scores: List[float]  # Track Mistral scores separately

    # Best result tracking
    best_joke: str
    best_score: float

    # Iteration tracking
    iteration_count: int
    max_iterations: int

    # History for debugging
    history: List[Dict]


In [ ]:

# ============================================================================
# NODE FUNCTIONS
# ============================================================================

def generate_jokes_llama(state: JokeGenerationState, llama_gen: RAGHumorGenerator) -> Dict:
    """Generate jokes using Llama model"""
    word1 = state["word1"]
    word2 = state["word2"]
    iteration = state.get("iteration_count", 0)

    mode = "headline" if not word2 else "word-pair"
    print(f"\n[Iteration {iteration + 1}] Generating jokes with Llama ({mode} mode)...")

    result_with_RAG = llama_gen.generate_with_rag(
        word1=word1,
        word2=word2,
        use_rag=True,  # with RAG
        num_candidates=3
    )

    result_without_RAG = llama_gen.generate_with_rag(
        word1=word1,
        word2=word2,
        use_rag=False,  # No RAG - just fine-tuned model
        num_candidates=3
    )

    jokes = result_with_RAG["generated_jokes"] + result_without_RAG["generated_jokes"]

    print('result_with_RAG jokes:')
    print(result_with_RAG["generated_jokes"])

    print('result_without_RAG jokes:')
    print(result_without_RAG["generated_jokes"])

    print(f"   Generated {len(jokes)} Llama jokes")

    return {"llama_jokes": jokes}


def generate_jokes_gpt(state: JokeGenerationState, gpt_gen: GPT4JokeGenerator) -> Dict:
    """Generate jokes using GPT-OSS (reduced to 2 to limit self-bias)"""
    word1 = state["word1"]
    word2 = state["word2"]
    iteration = state.get("iteration_count", 0)

    mode = "headline" if not word2 else "word-pair"
    print(f"\n[Iteration {iteration + 1}] Generating jokes with GPT-OSS ({mode} mode)...")

    jokes = gpt_gen.generate_jokes(word1, word2, num_jokes=2, verbose=False)

    print(f"   Generated {len(jokes)} GPT-OSS jokes")

    return {"gpt_jokes": jokes}


def combine_jokes(state: JokeGenerationState) -> JokeGenerationState:
    """Combine jokes from both generators and track sources"""
    llama_jokes = state.get("llama_jokes", [])
    gpt_jokes = state.get("gpt_jokes", [])

    all_jokes = llama_jokes + gpt_jokes
    sources = ["Llama"] * len(llama_jokes) + ["GPT"] * len(gpt_jokes)

    print(f"\nCombined {len(all_jokes)} total jokes ({len(llama_jokes)} Llama + {len(gpt_jokes)} GPT)")

    return {
        **state,
        "candidate_jokes": all_jokes,
        "joke_sources": sources,
    }


def normalize_mistral_score(raw: float) -> float:
    """
    Normalize Mistral humor score (0-4) to 0-10 scale using
    percentile-aware mapping based on training data distribution.
    
    Training data stats (hahackathon_train.csv):
      - Range: 0.1 - 4.0, Mean: 2.26, SD: 0.57
      - ~90% of scores fall between 1.5 and 3.0
      - Score 3.0+ = top quartile (genuinely funny)
      - Score 1.5-  = bottom quartile (not funny)
    
    We use a piecewise linear mapping that stretches the meaningful
    range (1.0-3.5) across most of the 0-10 scale:
      Mistral 0.0 -> 0.0    (no humor)
      Mistral 1.0 -> 2.0    (weak humor)
      Mistral 2.0 -> 4.5    (below average)
      Mistral 2.26-> 5.0    (average - matches training mean)
      Mistral 3.0 -> 7.0    (good humor)
      Mistral 3.5 -> 8.5    (very good)
      Mistral 4.0 -> 10.0   (exceptional)
    """
    raw = max(0.0, min(4.0, raw))
    
    # Piecewise linear mapping
    breakpoints = [(0.0, 0.0), (1.0, 2.0), (2.0, 4.5), (2.26, 5.0),
                   (3.0, 7.0), (3.5, 8.5), (4.0, 10.0)]
    
    for i in range(len(breakpoints) - 1):
        x0, y0 = breakpoints[i]
        x1, y1 = breakpoints[i + 1]
        if raw <= x1:
            t = (raw - x0) / (x1 - x0)
            return y0 + t * (y1 - y0)
    
    return 10.0


def score_jokes_dual(
    state: JokeGenerationState,
    llm_scorer: LLMHumorScorer,
    mistral_scorer: MistralHumorScorer
) -> JokeGenerationState:
    """
    Score jokes using dual evaluator ensemble.
    
    Scoring weights:
    - GPT-OSS (60%): Multi-criteria quality assessment. Scale 1-10.
    - Mistral (40%): Pure humor rating (0-4 scale, normalized to 0-10).
    
    GPT self-bias discount: GPT evaluation scores are multiplied by 0.80
    for GPT-generated jokes to counteract GPT rating its own output higher.
    """
    jokes = state["candidate_jokes"]
    word1 = state["word1"]
    word2 = state["word2"]
    sources = state.get("joke_sources", [])
    iteration = state.get("iteration_count", 0)

    print(f"\n[Iteration {iteration + 1}] Scoring {len(jokes)} jokes with GPT-OSS + Mistral...")

    # --- GPT-OSS: multi-criteria quality (1-10) ---
    print("   Scoring with GPT-OSS (batch mode - single API call)...")
    gpt_scores_raw = llm_scorer.score_jokes(jokes, word1, word2, verbose=False)

    # Apply GPT self-bias discount for GPT-generated jokes
    GPT_SELF_BIAS_DISCOUNT = 0.80
    gpt_scores = []
    for i, score in enumerate(gpt_scores_raw):
        src = sources[i] if i < len(sources) else "?"
        if src == "GPT":
            gpt_scores.append(score * GPT_SELF_BIAS_DISCOUNT)
        else:
            gpt_scores.append(score)

    # --- Mistral: pure humor rating (0-4) ---
    print("   Scoring with Mistral (humor funniness only)...")
    mistral_scores = mistral_scorer.score_jokes(jokes, word1, word2, verbose=False)
    
    # Normalize Mistral to 0-10 using distribution-aware mapping
    mistral_scores_normalized = [normalize_mistral_score(s) for s in mistral_scores]
    
    # Weighted ensemble: GPT 60% + Mistral 40%
    W_GPT = 0.6
    W_MISTRAL = 0.4
    
    final_scores = [
        W_GPT * gpt + W_MISTRAL * mistral
        for gpt, mistral in zip(gpt_scores, mistral_scores_normalized)
    ]

    # Print score breakdown
    print(f"\n   Score breakdown (GPT {W_GPT:.0%} + Mistral {W_MISTRAL:.0%}, GPT self-bias discount={GPT_SELF_BIAS_DISCOUNT}):")
    print(f"   {'='*120}")
    print(f"   {'#':>3} {'Src':<6} {'GPT_raw':>7} {'GPT_adj':>7}  {'Mistral':>7} {'(norm)':>7}  {'Final':>7}  Joke")
    print(f"   {'-'*120}")
    for i, (joke, gpt_raw, gpt_adj, m_raw, m_norm, final) in enumerate(
        zip(jokes, gpt_scores_raw, gpt_scores, mistral_scores, mistral_scores_normalized, final_scores)
    ):
        src = sources[i] if i < len(sources) else "?"
        disc = "*" if src == "GPT" else " "
        print(f"   {i+1:3d} {src:<6} {gpt_raw:5.2f}/10 {gpt_adj:5.2f}{disc}   {m_raw:5.2f}/4 ({m_norm:5.2f})  {final:5.2f}/10  {joke[:60]}...")

    # Find best joke
    if final_scores:
        best_idx = final_scores.index(max(final_scores))
        best_joke = jokes[best_idx]
        best_score = final_scores[best_idx]
    else:
        best_joke = ""
        best_score = 0.0

    print(f"\n   * = GPT score discounted by {GPT_SELF_BIAS_DISCOUNT} (GPT-generated joke)")
    print(f"\n   Best:  {best_score:.2f}/10 (GPT: {gpt_scores[best_idx]:.2f}, Mistral: {mistral_scores[best_idx]:.2f}/4)")
    print(f"   Avg:   {sum(final_scores)/len(final_scores):.2f}/10")
    print(f"   Range: {min(final_scores):.2f} - {max(final_scores):.2f}")

    return {
        **state,
        "scores": final_scores,
        "gpt_scores": gpt_scores,
        "mistral_scores": mistral_scores,
        "best_joke": best_joke,
        "best_score": best_score
    }


def check_score_threshold(state: JokeGenerationState) -> Literal["refine", "output"]:
    """Decision node: check if score meets threshold"""
    score = state["best_score"]
    iteration = state.get("iteration_count", 0)
    max_iter = state.get("max_iterations", 3)

    print(f"\nDecision: Score={score:.2f}, Iteration={iteration + 1}/{max_iter}")

    if score >= 7.0:
        print(f"   Score threshold met! (>=7.0)")
        return "output"
    elif iteration >= max_iter - 1:
        print(f"   Max iterations reached, outputting best result")
        return "output"
    else:
        print(f"   Score below threshold, refining...")
        return "refine"


def increment_iteration(state: JokeGenerationState) -> JokeGenerationState:
    """Increment iteration counter and save history"""
    current_iter = state.get("iteration_count", 0)
    history = state.get("history", [])

    # Save this iteration's results
    history.append({
        "iteration": current_iter + 1,
        "best_score": state["best_score"],
        "best_joke": state["best_joke"],
        "num_candidates": len(state.get("candidate_jokes", [])),
        "avg_score": sum(state.get("scores", [0])) / max(len(state.get("scores", [1])), 1),
        "avg_gpt_score": sum(state.get("gpt_scores", [0])) / max(len(state.get("gpt_scores", [1])), 1),
        "avg_mistral_score": sum(state.get("mistral_scores", [0])) / max(len(state.get("mistral_scores", [1])), 1)
    })

    return {
        **state,
        "iteration_count": current_iter + 1,
        "history": history
    }


def output_result(state: JokeGenerationState) -> JokeGenerationState:
    """Final output node - shows best and second-best joke with source"""
    print(f"\n{'='*80}")
    print(f"FINAL RESULT")
    print(f"{'='*80}")

    word1 = state['word1']
    word2 = state['word2']
    if word2:
        print(f"\nWord Pair: '{word1}' + '{word2}'")
    else:
        print(f"\nHeadline: '{word1}'")
    print(f"Total Iterations: {state.get('iteration_count', 0) + 1}")

    # Rank all jokes by final score
    jokes = state.get("candidate_jokes", [])
    scores = state.get("scores", [])
    gpt_sc = state.get("gpt_scores", [])
    mis_sc = state.get("mistral_scores", [])
    sources = state.get("joke_sources", [])

    if jokes and scores:
        ranked = sorted(zip(jokes, scores, gpt_sc, mis_sc, sources + ["?"] * max(0, len(jokes) - len(sources))),
                        key=lambda x: x[1], reverse=True)

        # Best joke
        j1, f1, g1, m1, s1 = ranked[0]
        print(f"\n  #1 BEST  [{f1:.2f}/10]  (GPT:{g1:.1f}, Mistral:{m1:.2f}/4)  Source:{s1}")
        print(f"  {j1}")

        # Second best
        if len(ranked) >= 2:
            j2, f2, g2, m2, s2 = ranked[1]
            print(f"\n  #2       [{f2:.2f}/10]  (GPT:{g2:.1f}, Mistral:{m2:.2f}/4)  Source:{s2}")
            print(f"  {j2}")
    else:
        print(f"\n  (no jokes scored)")

    print(f"\n{'='*80}\n")

    return state


In [ ]:


# ============================================================================
# GRAPH BUILDER
# ============================================================================

def build_joke_generation_graph(
    llama_generator: RAGHumorGenerator,
    gpt_generator: GPT4JokeGenerator,
    llm_scorer: LLMHumorScorer,
    mistral_scorer: MistralHumorScorer
):
    """
    Build the LangGraph for joke generation pipeline with dual evaluation.

    Args:
        llama_generator: RAGHumorGenerator
        gpt_generator: GPT4JokeGenerator
        llm_scorer: LLMHumorScorer
        mistral_scorer: MistralHumorScorer

    Returns:
        Compiled LangGraph
    """
    workflow = StateGraph(JokeGenerationState)

    # Add nodes with bound generators/scorers
    workflow.add_node(
        "start",
        lambda state: state  # pass-through to fan out to both generators
    )
    workflow.add_node(
        "generate_llama",
        lambda state: generate_jokes_llama(state, llama_generator)
    )
    workflow.add_node(
        "generate_gpt",
        lambda state: generate_jokes_gpt(state, gpt_generator)
    )
    workflow.add_node("combine_jokes", combine_jokes)
    workflow.add_node(
        "score_jokes",
        lambda state: score_jokes_dual(state, llm_scorer, mistral_scorer)
    )
    workflow.add_node("increment_iteration", increment_iteration)
    workflow.add_node("output", output_result)

    # Fan-out: start node dispatches to both generators in parallel
    workflow.set_entry_point("start")
    workflow.add_edge("start", "generate_llama")
    workflow.add_edge("start", "generate_gpt")

    # Add edges - both generators feed into combine
    workflow.add_edge("generate_llama", "combine_jokes")
    workflow.add_edge("generate_gpt", "combine_jokes")

    # After combining, score the jokes
    workflow.add_edge("combine_jokes", "score_jokes")

    # Decision point: refine or output
    workflow.add_conditional_edges(
        "score_jokes",
        check_score_threshold,
        {
            "refine": "increment_iteration",
            "output": "output"
        }
    )

    # Refinement loop - go back to start (fans out to both generators)
    workflow.add_edge("increment_iteration", "start")

    # End the workflow
    workflow.add_edge("output", END)

    return workflow.compile()


In [ ]:
# ============================================================================
# LOAD MODELS (run once, then skip when re-running pipeline)
# ============================================================================

print("="*80)
print("LANGGRAPH JOKE GENERATION PIPELINE - DUAL EVALUATOR MODE")
print("GPT-OSS (60%) + Mistral (40%) Ensemble Scoring")
print("="*80)
print()

from dotenv import load_dotenv
load_dotenv(Path("../../.env"))

NVIDIA_API_KEY = os.getenv("NVIDIA_API_KEY")
if not NVIDIA_API_KEY:
    raise ValueError("NVIDIA_API_KEY not found in environment. Please set it in .env file")

config = RAGConfig()
config.TEMPERATURE = 0.65
config.TOP_K = 50

print("Initializing components...\n")

# 1. Llama Generator
try:
    llama_gen = RAGHumorGenerator(config)
    llama_gen.load_model()
    print("Llama generator initialized\n")
except Exception as e:
    print(f"Could not load Llama model: {e}\n")
    llama_gen = None

# 2. GPT-OSS Generator
gpt_gen = GPT4JokeGenerator(
    api_key=NVIDIA_API_KEY,
    temperature=0.8,
    max_retries=1
)
print("GPT-OSS generator initialized\n")

# 3. GPT-OSS Scorer
llm_scorer = LLMHumorScorer(
    api_key=NVIDIA_API_KEY,
    temperature=0.4,
    max_retries=2
)
print("GPT-OSS scorer initialized\n")

# 4. Mistral Scorer (auto-detects v2 checkpoint with recovered head weights)
mistral_scorer = MistralHumorScorer(
    base_model_id="mistralai/Ministral-3-8B-Base-2512",
    num_labels=2,
    max_length=512
)
print(f"Mistral checkpoint: {mistral_scorer.checkpoint_path}")

if not mistral_scorer.checkpoint_path.exists():
    print(f"Mistral checkpoint not found! Expected: {mistral_scorer.checkpoint_path}")
    mistral_scorer = None
else:
    mistral_scorer.load_model()
    print("Mistral scorer initialized\n")

print("="*80)
print("All models loaded.")
print("="*80)

In [ ]:
# ============================================================================
# BUILD RAG DATABASE & COMPILE GRAPH (run once after loading models)
# ============================================================================

# Load RAG database
if llama_gen is not None:
    llama_gen.load_rag_database(build_if_missing=True)
    print("RAG database loaded\n")

# Build the graph
if mistral_scorer is None:
    raise RuntimeError("Cannot run dual evaluator mode without Mistral scorer")
if llama_gen is None:
    raise RuntimeError("Cannot run pipeline without Llama generator")

graph = build_joke_generation_graph(llama_gen, gpt_gen, llm_scorer, mistral_scorer)
print("LangGraph compiled (Dual Evaluator mode)")

In [ ]:

# ============================================================================
# RUN PIPELINE (re-run this cell to regenerate jokes)
# ============================================================================
import csv
from datetime import datetime

# Supports two input modes:
#   ("word1", "word2")  -> word-pair mode: joke connecting both words
#   ("headline text", "") -> headline mode: funny commentary on headline
test_pairs = [
    ("banana", "satellite"),
    ("angry", "teacup"),
    ("cosmic", "toaster"),
    ("sleepy", "volcano"),
    ("neon", "hamster"),
    ("awkward", "penguin"),
    ("electric", "spaghetti"),
    ("dramatic", "cactus"),
    ("confused", "astronaut"),
    ("rubber", "tornado"),
    ("invisible", "taco"),
    ("sarcastic", "dolphin"),
    ("haunted", "calculator"),
    ("spicy", "cloud"),
    ("polite", "chainsaw"),
    ("disco", "librarian"),
    ("lonely", "robot"),
    ("caffeinated", "squirrel"),
    ("confused", "mailbox"),
    # Headline mode examples (word2 = ""):
    # ("Scientists discover that cats have been faking sleep for centuries", ""),
    # ("Local man wins lottery, immediately buys 10000 rubber ducks", ""),
]

results_summary = []

# --- Open CSVs BEFORE the loop so results are saved incrementally ---
output_dir = PROJECT_ROOT / "output"
output_dir.mkdir(exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

best_csv_path = output_dir / f"best_jokes_{timestamp}.csv"
all_csv_path  = output_dir / f"all_jokes_{timestamp}.csv"

best_csv_file = open(best_csv_path, 'w', newline='', encoding='utf-8')
all_csv_file  = open(all_csv_path,  'w', newline='', encoding='utf-8')

best_writer = csv.writer(best_csv_file)
all_writer  = csv.writer(all_csv_file)

best_writer.writerow(["#", "Word1", "Word2", "Best_Joke", "Best_Source", "2nd_Best_Joke", "2nd_Source",
                       "Best_GPT", "Best_Mistral_0to4", "Best_Final",
                       "2nd_GPT",  "2nd_Mistral_0to4",  "2nd_Final", "Human_Rating"])
all_writer.writerow(["#", "PairIdx", "Rank", "Word1", "Word2", "Joke", "Source",
                      "GPT_Score", "Mistral_0to4", "Mistral_Norm", "Final_0to10", "Human_Rating"])

all_row_num = 1
print(f"Incremental CSVs: {best_csv_path.name} / {all_csv_path.name}")

for pair_idx, (word1, word2) in enumerate(test_pairs, 1):
    is_headline = not word2
    if is_headline:
        print("\n" + "="*80)
        print(f"PROCESSING [{pair_idx}/{len(test_pairs)}] HEADLINE: '{word1}'")
        print("="*80)
    else:
        print("\n" + "="*80)
        print(f"PROCESSING [{pair_idx}/{len(test_pairs)}]: '{word1}' + '{word2}'")
        print("="*80)

    initial_state = {
        "word1": word1, "word2": word2,
        "llama_jokes": [], "gpt_jokes": [], "candidate_jokes": [],
        "joke_sources": [],
        "scores": [], "gpt_scores": [], "mistral_scores": [],
        "best_joke": "", "best_score": 0.0,
        "iteration_count": 0, "max_iterations": 1, "history": []
    }

    try:
        result = graph.invoke(initial_state)
        total_candidates = len(result.get('candidate_jokes', []))

        label = f"Headline: {word1}" if is_headline else f"{word1} + {word2}"
        r = {
            "word_pair": label,
            "word1": word1,
            "word2": word2,
            "best_score": result['best_score'],
            "best_joke": result['best_joke'],
            "all_jokes": result.get('candidate_jokes', []),
            "all_scores": result.get('scores', []),
            "gpt_scores": result.get('gpt_scores', []),
            "mistral_scores": result.get('mistral_scores', []),
            "joke_sources": result.get('joke_sources', []),
            "iterations": result.get('iteration_count', 0) + 1,
            "total_candidates": total_candidates
        }
        results_summary.append(r)

        # --- Write to CSVs immediately after each word pair ---
        sources = r['joke_sources']
        jd = list(zip(r['all_jokes'], r['all_scores'], r['gpt_scores'], r['mistral_scores'],
                       sources + ["?"] * max(0, len(r['all_jokes']) - len(sources))))
        jd.sort(key=lambda x: x[1], reverse=True)

        # Best + 2nd best row
        if len(jd) >= 2:
            j1, f1, g1, m1, s1 = jd[0]
            j2, f2, g2, m2, s2 = jd[1]
        elif len(jd) == 1:
            j1, f1, g1, m1, s1 = jd[0]
            j2, f2, g2, m2, s2 = "", 0, 0, 0, ""
        else:
            j1, f1, g1, m1, s1 = "", 0, 0, 0, ""
            j2, f2, g2, m2, s2 = "", 0, 0, 0, ""

        best_writer.writerow([
            pair_idx, word1, word2,
            j1, s1, j2, s2,
            f"{g1:.2f}", f"{m1:.2f}", f"{f1:.2f}",
            f"{g2:.2f}", f"{m2:.2f}", f"{f2:.2f}", ""
        ])
        best_csv_file.flush()

        # All jokes for this pair
        for rank, (joke, final, gpt, mistral, src) in enumerate(jd, 1):
            m_norm = normalize_mistral_score(mistral)
            all_writer.writerow([
                all_row_num, pair_idx, rank, word1, word2, joke, src,
                f"{gpt:.2f}", f"{mistral:.2f}", f"{m_norm:.2f}", f"{final:.2f}", ""
            ])
            all_row_num += 1
        all_csv_file.flush()

    except Exception as e:
        print(f"\nError processing word pair: {e}")
        import traceback
        traceback.print_exc()

best_csv_file.close()
all_csv_file.close()

print(f"\nDone! Generated jokes for {len(results_summary)}/{len(test_pairs)} entries.")
print(f"Best+2nd CSV : {best_csv_path}")
print(f"All jokes CSV: {all_csv_path}  ({all_row_num - 1} jokes)")


In [ ]:

# ============================================================================
# FULL OUTPUT FOR HUMAN RATING (re-run to reformat without regenerating)
# ============================================================================

print("#"*80)
print("#" + " "*26 + "JOKES FOR HUMAN RATING" + " "*30 + "#")
print("#"*80)
print(f"\nTotal entries: {len(results_summary)}")
print(f"Total jokes generated: {sum(r['total_candidates'] for r in results_summary)}")
print(f"Scoring: GPT-OSS 60% (multi-criteria) + Mistral 40% (humor only)")
print(f"GPT self-bias discount: 0.80 for GPT-generated jokes")
print()

for pair_idx, r in enumerate(results_summary, 1):
    print("="*80)
    if r['word2']:
        print(f"WORD PAIR {pair_idx}/{len(results_summary)}: '{r['word1']}' + '{r['word2']}'")
    else:
        print(f"HEADLINE {pair_idx}/{len(results_summary)}: '{r['word1']}'")
    print(f"Best Score: {r['best_score']:.2f}/10")
    print("="*80)

    # Sort jokes by final score (best first)
    sources = r.get('joke_sources', [])
    joke_data = list(zip(
        r['all_jokes'], r['all_scores'], r['gpt_scores'], r['mistral_scores'],
        sources + ["?"] * max(0, len(r['all_jokes']) - len(sources))
    ))
    joke_data.sort(key=lambda x: x[1], reverse=True)

    for j_idx, (joke, final, gpt, mistral, src) in enumerate(joke_data, 1):
        mistral_norm = normalize_mistral_score(mistral)
        is_best = " << SELECTED" if joke == r['best_joke'] else ""
        print(f"\n  [{j_idx:2d}] Score: {final:.2f}/10  (GPT: {gpt:.1f}, Mistral: {mistral:.2f}/4 -> {mistral_norm:.1f}/10)  Src:{src}{is_best}")
        print(f"  {'-'*70}")
        print(f"  {joke}")

    print()

# --- Summary table ---
print("="*80)
print("RESULTS SUMMARY")
print("="*80)
print(f"\n{'#':<4} {'Input':<35} {'Score':<8} {'GPT':<8} {'Mistral':<8} {'BestSrc':<8} {'Jokes':<8}")
print("-"*85)
for i, r in enumerate(results_summary, 1):
    avg_gpt = sum(r['gpt_scores']) / len(r['gpt_scores']) if r['gpt_scores'] else 0
    avg_mis = sum(r['mistral_scores']) / len(r['mistral_scores']) if r['mistral_scores'] else 0
    label = r['word_pair'][:33]
    # Find source of best joke
    sources = r.get('joke_sources', [])
    jokes = r['all_jokes']
    scores = r['all_scores']
    best_src = "?"
    if jokes and scores:
        best_idx = scores.index(max(scores))
        best_src = sources[best_idx] if best_idx < len(sources) else "?"
    print(f"{i:<4} {label:<35} {r['best_score']:<8.2f} {avg_gpt:<8.1f} {avg_mis:<8.2f} {best_src:<8} {r['total_candidates']:<8}")

avg_score = sum(r['best_score'] for r in results_summary) / len(results_summary) if results_summary else 0
print("-"*85)
print(f"{'':4} {'AVERAGE':<35} {avg_score:<8.2f}")

# --- Save JSON (full data for programmatic access) ---
output_dir = PROJECT_ROOT / "output"
output_dir.mkdir(exist_ok=True)
json_path = output_dir / f"jokes_for_rating_{timestamp}.json"
with open(json_path, 'w', encoding='utf-8') as f:
    json.dump(results_summary, f, indent=2, ensure_ascii=False)
print(f"\nJSON saved: {json_path}")
print(f"CSVs already saved incrementally during pipeline run:")
print(f"  Best+2nd : {best_csv_path}")
print(f"  All jokes: {all_csv_path}")
print(f"\n  -> Open CSVs in Excel and fill 'Human_Rating' column (1-10)")
print(f"\n{'='*80}")
print("Pipeline complete!")
print("="*80)
